In [1]:
import tensorflow as tf

2024-08-06 10:44:43.096710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 10:44:43.145033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 10:44:43.159036: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 10:44:43.198251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 10:44:45.641321: W tensorflow/compiler/tf2

In [2]:
if tf.config.list_physical_devices('GPU'):
  print("GPU is available")
  print("CUDA Available: ", tf.config.list_physical_devices('GPU'))
  
else:
  print("GPU is not available")

GPU is available
CUDA Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1722930289.722313   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930289.794976   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930289.795120   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


In [4]:
import pandas as pd
import os

In [5]:
cwd = os.getcwd()
timeseries_df = pd.read_csv(cwd + "/datasets/timeseries_with_wav.csv")
timeseries_df = timeseries_df.drop('Unnamed: 0', axis=1)

In [6]:
timeseries_df

,Millis,ECG,IR,RED,patient,datetime,rate_avg,rate_std,rate_min,rate_max
0,892,226.0,28556.0,28966.0,1,2023-10-18 09:38:39,0.000000,0.000000,0,0
1,893,226.0,28554.0,28959.0,1,2023-10-18 09:38:39,0.000000,0.000000,0,0
2,893,226.0,28556.0,28960.0,1,2023-10-18 09:38:39,0.000000,0.000000,0,0
3,893,226.0,28552.0,28959.0,1,2023-10-18 09:38:39,0.000000,0.000000,0,0
4,894,226.0,28551.0,28954.0,1,2023-10-18 09:38:39,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...
547272,157,219.0,26252.0,29845.0,41,2023-10-27 15:56:43,57.552448,797.614263,-1534,911
547273,168,219.0,26254.0,29843.0,41,2023-10-27 15:56:43,550.286713,1328.394839,-1538,2144
547274,170,219.0,26252.0,29844.0,41,2023-10-27 15:56:43,1184.440559,257.112438,602,1880
547275,172,219.0,26250.0,29843.0,41,2023-10-27 15:56:43,-1232.818182,903.267175,-2660,565


In [7]:
timeseries_df = timeseries_df.drop(columns=['datetime'])

In [8]:
result_df = pd.read_csv(cwd + "/datasets/patient.csv")
result_df = result_df.drop('Unnamed: 0', axis=1)
filtered_df = result_df[(result_df['result'] == 'LA') | (result_df['result'] == 'LA DİLATASYONU')]
filtered_df.loc[:, 'result'] = filtered_df['result'].apply(lambda x: 0 if x == 'LA' else 1)
timeseries_df = pd.merge(timeseries_df, filtered_df, on='patient')

In [9]:
timeseries_df

,Millis,ECG,IR,RED,patient,rate_avg,rate_std,rate_min,rate_max,result
0,892,226.0,28556.0,28966.0,1,0.000000,0.000000,0,0,1
1,893,226.0,28554.0,28959.0,1,0.000000,0.000000,0,0,1
2,893,226.0,28556.0,28960.0,1,0.000000,0.000000,0,0,1
3,893,226.0,28552.0,28959.0,1,0.000000,0.000000,0,0,1
4,894,226.0,28551.0,28954.0,1,0.000000,0.000000,0,0,1
...,...,...,...,...,...,...,...,...,...,...
547272,157,219.0,26252.0,29845.0,41,57.552448,797.614263,-1534,911,1
547273,168,219.0,26254.0,29843.0,41,550.286713,1328.394839,-1538,2144,1
547274,170,219.0,26252.0,29844.0,41,1184.440559,257.112438,602,1880,1
547275,172,219.0,26250.0,29843.0,41,-1232.818182,903.267175,-2660,565,1


In [10]:
personal_df = pd.read_csv(cwd + "/datasets/patient_info.csv")
personal_df = personal_df.drop('Unnamed: 0', axis=1)
personal_df

,id,name_surname,identification_number,date,sex,HT,DM,HL,KAH,KOAH,KBY,KC HST,NOT,complaint,height,weight,systolic_pressure,diastolic_pressure
0,1,NURCAN TANTÜR,63115237952,2023-10-18,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,158.0,95.0,141.0,77.0
1,2,MÜNÜR ÇAPKINLAR,34333168786,2023-10-18,E,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,170.0,83.0,132.0,80.0
2,3,VEZİR YAŞAR,31936213592,2023-10-18,E,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,165.0,73.0,39.0,77.0
3,4,MENŞURE ÇİNİCİ,25627707118,2023-10-18,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,78.0,146.0,90.0
4,5,OĞUZHAN GÜLOĞULLARI,20023624076,2023-10-18,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,ATGA,178.0,70.0,32.0,62.0
5,6,SENFİNAZ ARAS,17821733192,2023-10-18,K,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,155.0,81.0,45.0,86.0
6,7,SEVDA UYSAL,14605798910,2023-10-18,K,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,143.0,75.0,186.0,95.0
7,8,LÜTFİYE ADALI,18853683912,2023-10-18,K,0.0,0.0,0.0,1.0,0.0,0.0,0.0,AVR+MVR,NaN,156.0,70.0,137.0,64.0
8,9,SELAMİ KOŞUCU,24565459066,2023-10-18,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,175.0,75.0,131.0,83.0
9,10,KADİR YALÇIN,33826072180,2023-10-19,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,171.0,60.0,113.0,74.0


In [11]:
mean_weight = personal_df[personal_df.sex == 'K']['weight'].mean()
mean_height = personal_df[personal_df.sex == 'K']['height'].mean()


In [12]:

personal_df.loc[personal_df.id == 4, 'height'] = mean_height
personal_df.loc[personal_df.id == 30, 'weight'] = mean_weight
personal_df.loc[personal_df.id == 30, 'height'] = mean_height

In [13]:
personal_df

,id,name_surname,identification_number,date,sex,HT,DM,HL,KAH,KOAH,KBY,KC HST,NOT,complaint,height,weight,systolic_pressure,diastolic_pressure
0,1,NURCAN TANTÜR,63115237952,2023-10-18,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,158.000000,95.000000,141.0,77.0
1,2,MÜNÜR ÇAPKINLAR,34333168786,2023-10-18,E,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,170.000000,83.000000,132.0,80.0
2,3,VEZİR YAŞAR,31936213592,2023-10-18,E,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,165.000000,73.000000,39.0,77.0
3,4,MENŞURE ÇİNİCİ,25627707118,2023-10-18,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,157.181818,78.000000,146.0,90.0
4,5,OĞUZHAN GÜLOĞULLARI,20023624076,2023-10-18,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,ATGA,178.000000,70.000000,32.0,62.0
5,6,SENFİNAZ ARAS,17821733192,2023-10-18,K,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,155.000000,81.000000,45.0,86.0
6,7,SEVDA UYSAL,14605798910,2023-10-18,K,1.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,143.000000,75.000000,186.0,95.0
7,8,LÜTFİYE ADALI,18853683912,2023-10-18,K,0.0,0.0,0.0,1.0,0.0,0.0,0.0,AVR+MVR,NaN,156.000000,70.000000,137.0,64.0
8,9,SELAMİ KOŞUCU,24565459066,2023-10-18,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,175.000000,75.000000,131.0,83.0
9,10,KADİR YALÇIN,33826072180,2023-10-19,E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,171.000000,60.000000,113.0,74.0


In [14]:
mean_sys_w = personal_df[personal_df.sex == 'K']['systolic_pressure'].mean()
mean_dia_W = personal_df[personal_df.sex == 'K']['diastolic_pressure'].mean()

mean_sys_m = personal_df[personal_df.sex == 'E']['systolic_pressure'].mean()
mean_dia_m = personal_df[personal_df.sex == 'E']['diastolic_pressure'].mean()


personal_df.loc[personal_df.id == 33, 'systolic_pressure'] = mean_sys_w
personal_df.loc[personal_df.id == 33, 'diastolic_pressure'] = mean_dia_W

personal_df.loc[personal_df.id == 36, 'systolic_pressure'] = mean_sys_m
personal_df.loc[personal_df.id == 36, 'diastolic_pressure'] = mean_dia_m

In [15]:
personal_df.isna().sum()

id                        0
name_surname              0
identification_number     0
date                      0
sex                       0
HT                        1
DM                        1
HL                        1
KAH                       1
KOAH                      1
KBY                       1
KC HST                    1
NOT                      32
complaint                20
height                    0
weight                    0
systolic_pressure         0
diastolic_pressure        0
dtype: int64

In [16]:
personal_df.columns

Index(['id', 'name_surname', 'identification_number', 'date', 'sex', 'HT',
       'DM', 'HL', 'KAH', 'KOAH', 'KBY', 'KC HST', 'NOT', 'complaint',
       'height', 'weight', 'systolic_pressure', 'diastolic_pressure'],
      dtype='object')

In [17]:
timeseries_df

,Millis,ECG,IR,RED,patient,rate_avg,rate_std,rate_min,rate_max,result
0,892,226.0,28556.0,28966.0,1,0.000000,0.000000,0,0,1
1,893,226.0,28554.0,28959.0,1,0.000000,0.000000,0,0,1
2,893,226.0,28556.0,28960.0,1,0.000000,0.000000,0,0,1
3,893,226.0,28552.0,28959.0,1,0.000000,0.000000,0,0,1
4,894,226.0,28551.0,28954.0,1,0.000000,0.000000,0,0,1
...,...,...,...,...,...,...,...,...,...,...
547272,157,219.0,26252.0,29845.0,41,57.552448,797.614263,-1534,911,1
547273,168,219.0,26254.0,29843.0,41,550.286713,1328.394839,-1538,2144,1
547274,170,219.0,26252.0,29844.0,41,1184.440559,257.112438,602,1880,1
547275,172,219.0,26250.0,29843.0,41,-1232.818182,903.267175,-2660,565,1


In [18]:
df = pd.merge(timeseries_df, personal_df.drop(columns=['name_surname', 'date', 'NOT', 'complaint', 'identification_number', 'systolic_pressure',	'diastolic_pressure']), left_on='patient', right_on='id')
df = df.drop('id', axis=1)
df


,Millis,ECG,IR,RED,patient,rate_avg,rate_std,rate_min,rate_max,result,sex,HT,DM,HL,KAH,KOAH,KBY,KC HST,height,weight
0,892,226.0,28556.0,28966.0,1,0.000000,0.000000,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
1,893,226.0,28554.0,28959.0,1,0.000000,0.000000,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
2,893,226.0,28556.0,28960.0,1,0.000000,0.000000,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
3,893,226.0,28552.0,28959.0,1,0.000000,0.000000,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
4,894,226.0,28551.0,28954.0,1,0.000000,0.000000,0,0,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,158.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547272,157,219.0,26252.0,29845.0,41,57.552448,797.614263,-1534,911,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,105.0
547273,168,219.0,26254.0,29843.0,41,550.286713,1328.394839,-1538,2144,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,105.0
547274,170,219.0,26252.0,29844.0,41,1184.440559,257.112438,602,1880,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,105.0
547275,172,219.0,26250.0,29843.0,41,-1232.818182,903.267175,-2660,565,1,K,1.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,105.0


In [19]:
df.isna().sum()

Millis          0
ECG             0
IR              0
RED             0
patient         0
rate_avg        0
rate_std        0
rate_min        0
rate_max        0
result          0
sex             0
HT          17083
DM          17083
HL          17083
KAH         17083
KOAH        17083
KBY         17083
KC HST      17083
height          0
weight          0
dtype: int64

In [20]:
# the patient has not any info for other diseases
df = df.dropna()

In [21]:
df.result = df.result.astype('int')

/tmp/ipykernel_12165/265399608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.result = df.result.astype('int')


In [22]:
df.isna().sum()

Millis      0
ECG         0
IR          0
RED         0
patient     0
rate_avg    0
rate_std    0
rate_min    0
rate_max    0
result      0
sex         0
HT          0
DM          0
HL          0
KAH         0
KOAH        0
KBY         0
KC HST      0
height      0
weight      0
dtype: int64

In [23]:
df.dtypes

Millis        int64
ECG         float64
IR          float64
RED         float64
patient       int64
rate_avg    float64
rate_std    float64
rate_min      int64
rate_max      int64
result        int64
sex          object
HT          float64
DM          float64
HL          float64
KAH         float64
KOAH        float64
KBY         float64
KC HST      float64
height      float64
weight      float64
dtype: object

In [24]:
df.sex = df.sex.astype('category')
df.sex = df.sex.cat.codes

/tmp/ipykernel_12165/1245972726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sex = df.sex.astype('category')
/tmp/ipykernel_12165/1245972726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sex = df.sex.cat.codes


In [25]:
df.columns

Index(['Millis', 'ECG', 'IR', 'RED', 'patient', 'rate_avg', 'rate_std',
       'rate_min', 'rate_max', 'result', 'sex', 'HT', 'DM', 'HL', 'KAH',
       'KOAH', 'KBY', 'KC HST', 'height', 'weight'],
      dtype='object')

In [26]:
df = df[["Millis", "ECG", "IR", "RED", "patient", "result"]]

In [27]:
from sklearn.preprocessing import StandardScaler
features = ['ECG', 'IR', 'RED']

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])
df

,Millis,ECG,IR,RED,patient,result
0,892,0.601770,0.329916,-0.081553,1,1
1,893,0.601770,0.329398,-0.085284,1,1
2,893,0.601770,0.329916,-0.084751,1,1
3,893,0.601770,0.328879,-0.085284,1,1
4,894,0.601770,0.328620,-0.087949,1,1
...,...,...,...,...,...,...
547272,157,-0.745524,-0.267420,0.386950,41,1
547273,168,-0.745524,-0.266901,0.385884,41,1
547274,170,-0.745524,-0.267420,0.386417,41,1
547275,172,-0.745524,-0.267938,0.385884,41,1


In [28]:
n_classes = len(np.unique(df.result))

In [29]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler

In [30]:
import tensorflow.keras as keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [31]:
y_true_list = []
y_pred_list = []
y_pred_bool_list = []
model_list = []

In [32]:
timesteps = 4096
features = 5

In [33]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

timesteps = 32768
features = 19

# Assuming df is your DataFrame and build_model is your model building function
patients = df['patient'].unique()
np.random.shuffle(patients)
kf = KFold(n_splits=len(patients) // 8)

for train_index, test_index in kf.split(patients):
    train_patients = patients[train_index]
    test_patients = patients[test_index]

    train_data = df[df['patient'].isin(train_patients)]
    test_data = df[df['patient'].isin(test_patients)]

    train_X = train_data.drop('result', axis=1)
    train_y = train_data.result
    test_X = test_data.drop('result', axis=1)
    test_y = test_data.result

    # Ensure train_y and test_y are binary (0 or 1)
    train_y = train_y.astype(np.float32)
    test_y = test_y.astype(np.float32)
    
    train_X = train_X.values.reshape((train_X.values.shape[0], train_X.values.shape[1], 1))
    test_X = test_X.values.reshape((test_X.values.shape[0], test_X.values.shape[1], 1))
    
    model = Sequential()
    model.add(LSTM(1000, input_shape=(timesteps, features), return_sequences=True))
    model.add(LSTM(1000))
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Training the model
    model.fit(train_X, train_y, epochs=10, batch_size=4096)

    # Predicting on the same data (for simplicity)
    predictions = model.predict(test_X)
    predictions = (predictions > 0.5).astype(int)

    # Evaluating the model
    loss, accuracy = model.evaluate(test_X, test_y)
    print(f'Loss: {loss}, Accuracy: {accuracy}')

    # Generating the confusion matrix
    cm = confusion_matrix(test_y, predictions)
    print('Confusion Matrix:')
    print(cm)

I0000 00:00:1722930294.796459   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930294.796798   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930294.796907   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930295.247391   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722930295.247551   12165 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-06

Epoch 1/10


2024-08-06 10:45:04.170202: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 24/106 ━━━━━━━━━━━━━━━━━━━━ 30s 369ms/step - accuracy: 0.5309 - loss: 0.8147

In [32]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report



# Assuming df is your DataFrame and build_model is your model building function
patients = df['patient'].unique()
np.random.shuffle(patients)
kf = KFold(n_splits=len(patients) // 8)

for train_index, test_index in kf.split(patients):
    train_patients = patients[train_index]
    test_patients = patients[test_index]

    train_data = df[df['patient'].isin(train_patients)]
    test_data = df[df['patient'].isin(test_patients)]

    train_X = train_data.drop('result', axis=1)
    train_y = train_data.result
    test_X = test_data.drop('result', axis=1)
    test_y = test_data.result
    
    # Ensure train_y and test_y are binary (0 or 1)
    train_y = train_y.astype(np.float32)
    test_y = test_y.astype(np.float32)
    
    train_X = train_X.values.reshape((train_X.values.shape[0], train_X.values.shape[1], 1))
    test_X = test_X.values.reshape((test_X.values.shape[0], test_X.values.shape[1], 1))


    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    input_shape = train_X.shape[1:]
    model = build_model(
        input_shape,
        head_size=4096,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=4,
        mlp_units=[4096, 128, 128, 64, 16, 4],
        mlp_dropout=0.4,
        dropout=0.3,
    )
    
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["binary_crossentropy"],
    )

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    model.fit(
        train_X,
        train_y,
        epochs=100,
        batch_size=4096,
        callbacks=callbacks,
    )

    y_pred = model.predict(test_X, batch_size=4096, verbose=1)

    # Convert predictions and true labels back to binary class labels
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(test_y, axis=1)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

    # Print confusion matrix and classification report
    print("Confusion Matrix:")
    print(conf_matrix)

    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_pred_classes))

    


Epoch 1/100


2024-08-02 15:55:16.311040: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-08-02 15:55:16.351088: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 : RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5117050880 bytes.
2024-08-02 15:55:16.351183: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5117050880 bytes.
	 [[{{node StatefulPartitionedCall}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.



ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/honorsea/miniconda3/envs/tf/lib/python3.10/asyncio/base_events.py", li

In [33]:
for i in range(5):
    print("----------------------",i, "----------------------")
    print(classification_report(y_true_list[i], y_pred_bool_list[i]))

    

---------------------- 0 ----------------------
              precision    recall  f1-score   support

           0       0.04      0.00      0.00     39827
           1       0.66      1.00      0.80     78459

    accuracy                           0.66    118286
   macro avg       0.35      0.50      0.40    118286
weighted avg       0.45      0.66      0.53    118286

---------------------- 1 ----------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     16959
           1       0.83      1.00      0.91     82118

    accuracy                           0.83     99077
   macro avg       0.41      0.50      0.45     99077
weighted avg       0.69      0.83      0.75     99077

---------------------- 2 ----------------------
              precision    recall  f1-score   support

           0       0.93      0.00      0.00     13863
           1       0.87      1.00      0.93     89125

    accuracy                          

/home/honorsea/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honorsea/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honorsea/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honorsea/.

In [34]:
y_pred = model.predict(test_X, batch_size=2048, verbose=1)
    y_pred_bool = (y_pred > 0.5).astype(int).flatten()  # Convert to binary predictions

    y_true_list.append(test_y)
    y_pred_list.append(y_pred) 
    y_pred_bool_list.append(y_pred_bool)
    model_list.append(model)

    print(classification_report(test_y, y_pred_bool))

array([[0.43182236, 0.5681776 ],
       [0.43181336, 0.5681867 ],
       [0.4318042 , 0.5681958 ],
       ...,
       [0.41876897, 0.58123106],
       [0.4189586 , 0.5810414 ],
       [0.4173369 , 0.5826631 ]], dtype=float32)

In [37]:
from tensorflow.keras.models import save_model

In [34]:
model.save('model1.keras')

In [64]:
model1 = keras.models.load_model('model1.keras')


2024-07-25 15:17:01.068504: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 15:17:01.069519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 15:17:01.070383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

InternalError: cudaSetDevice() on GPU:0 failed. Status: CUDA-capable device(s) is/are busy or unavailable